# Ejercicio 1: Análisis Exploratorio de Datos

In [1]:
#%pip install pandas
#%pip install numpy
#%pip install matplotlib
#%pip install statsmodels
#%pip install tensorflow
#%pip install keras
#%pip install scikit-learn
#%pip install distutils
#%pip install openpyxl


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.tsa as tsa
import statsmodels as sm
from datetime import datetime
import os
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller

#Tensorflow
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential,save_model,load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.callbacks import EarlyStopping
from keras.metrics import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error
import keras

%matplotlib inline

In [3]:
precios_2021 = pd.read_excel('PRECIOS 2021.xlsx', engine='openpyxl')
precios_2022 = pd.read_excel('PRECIOS 2022.xlsx', engine='openpyxl')
precios_2023 = pd.read_excel('PRECIOS 2023.xlsx', engine='openpyxl')
precios_2024 = pd.read_excel('PRECIOS 2024.xlsx', engine='openpyxl')

consumo = pd.read_excel('CONSUMO.xlsx', engine='openpyxl')
importacion = pd.read_excel('IMPORTACION.xlsx', engine='openpyxl')

In [4]:
# Unificar datos de precios
precios = pd.concat([precios_2021, precios_2022, precios_2023, precios_2024], ignore_index=True)

# Convertir columnas de fecha a datetime
consumo['Fecha'] = pd.to_datetime(consumo['Fecha'], format='%b/%Y')
importacion['Fecha'] = pd.to_datetime(importacion['Fecha'], format='%b/%Y')
precios['FECHA'] = pd.to_datetime(precios['FECHA'], format='%d/%b/%Y')

In [5]:
consumo_seleccionado = consumo[['Fecha', 'Gasolina regular', 'Gasolina superior', 'Diesel bajo azufre', 'Gas licuado de petróleo']]
importacion_seleccionado = importacion[['Fecha', 'Gasolina regular', 'Gasolina superior', 'Diesel bajo azufre', 'Gas licuado de petróleo']]
precios_seleccionado = precios[['FECHA', 'Superior', 'Regular', 'Diesel', 'Glp Cilindro 25Lbs.']]


In [6]:
consumo_seleccionado.set_index('Fecha', inplace=True)
importacion_seleccionado.set_index('Fecha', inplace=True)
precios_seleccionado.set_index('FECHA', inplace=True)

In [7]:
# Selección de series
serie_gasolina_super = consumo_seleccionado['Gasolina superior']
serie_gasolina_regular = consumo_seleccionado['Gasolina regular']
serie_diesel = consumo_seleccionado['Diesel bajo azufre']
serie_gas_lp = consumo_seleccionado['Gas licuado de petróleo']

# Series de importación
serie_gasolina_super_import = importacion_seleccionado['Gasolina superior']
serie_gasolina_regular_import = importacion_seleccionado['Gasolina regular']
serie_diesel_import = importacion_seleccionado['Diesel bajo azufre']
serie_gas_lp_import = importacion_seleccionado['Gas licuado de petróleo']

# Series de precios
serie_gasolina_super_precios = precios_seleccionado['Superior']
serie_gasolina_regular_precios = precios_seleccionado['Regular']
serie_diesel_precios = precios_seleccionado['Diesel']
serie_gas_lp_precios = precios_seleccionado['Glp Cilindro 25Lbs.']


## Predicciones 2024 con los modelos

In [8]:
# Quitar los datos de 2024 para abajo de las series de tiempo
# Los test seran de 2024 en adelante
serie_gasolina_super_test = serie_gasolina_super['2021-01-01':]
serie_gasolina_regular_test = serie_gasolina_regular['2021-01-01':]
serie_diesel_test = serie_diesel['2021-01-01':]
serie_gas_lp_test = serie_gas_lp['2021-01-01':]

# Eliminar NaNs
serie_gasolina_super_test = serie_gasolina_super_test.dropna()
serie_gasolina_regular_test = serie_gasolina_regular_test.dropna()
serie_diesel_test = serie_diesel_test.dropna()
serie_gas_lp_test = serie_gas_lp_test.dropna()

# Obtener los datos de validation
serie_gasolina_super_validation = serie_gasolina_super['2020-01-01':'2020-12-31']
serie_gasolina_regular_validation = serie_gasolina_regular['2020-01-01':'2020-12-31']
serie_diesel_validation = serie_diesel['2020-01-01':'2020-12-31']
serie_gas_lp_validation = serie_gas_lp['2020-01-01':'2020-12-31']


# Remplazamos las series por las que tienen los datos de 2024 para abajo
serie_gasolina_super = serie_gasolina_super[:'2019-12-31']
serie_gasolina_regular = serie_gasolina_regular[:'2019-12-31']
serie_diesel = serie_diesel[:'2019-12-31']
serie_gas_lp = serie_gas_lp[:'2019-12-31']





In [9]:
# Quitar los datos de 2024 para abajo de las series de importación
# Los test seran de 2024 en adelante
serie_gasolina_super_import_test = serie_gasolina_super_import['2021-01-01':]
serie_gasolina_regular_import_test = serie_gasolina_regular_import['2021-01-01':]
serie_diesel_import_test = serie_diesel_import['2021-01-01':]
serie_gas_lp_import_test = serie_gas_lp_import['2021-01-01':]

# Eliminar NaNs
serie_gasolina_super_import_test = serie_gasolina_super_import_test.dropna()
serie_gasolina_regular_import_test = serie_gasolina_regular_import_test.dropna()
serie_diesel_import_test = serie_diesel_import_test.dropna()
serie_gas_lp_import_test = serie_gas_lp_import_test.dropna()

# Obtener los datos de validation
serie_gasolina_super_import_validation = serie_gasolina_super_import['2020-01-01':'2020-12-31']
serie_gasolina_regular_import_validation = serie_gasolina_regular_import['2020-01-01':'2020-12-31']
serie_diesel_import_validation = serie_diesel_import['2020-01-01':'2020-12-31']
serie_gas_lp_import_validation = serie_gas_lp_import['2020-01-01':'2020-12-31']


# Remplazamos las series por las que tienen los datos de 2024 para abajo
serie_gasolina_super_import = serie_gasolina_super_import[:'2019-12-31']
serie_gasolina_regular_import = serie_gasolina_regular_import[:'2019-12-31']
serie_diesel_import = serie_diesel_import[:'2019-12-31']
serie_gas_lp_import = serie_gas_lp_import[:'2019-12-31']

In [10]:
# Quitar los datos de 2024 para abajo de las series de precios
# Los test seran de 2024 en adelante
serie_gasolina_super_precios_test = serie_gasolina_super_precios['2024-01-01':]
serie_gasolina_regular_precios_test = serie_gasolina_regular_precios['2024-01-01':]
serie_diesel_precios_test = serie_diesel_precios['2024-01-01':]
serie_gas_lp_precios_test = serie_gas_lp_precios['2024-01-01':]

# Eliminar NaNs
serie_gasolina_super_precios_test = serie_gasolina_super_precios_test.dropna()
serie_gasolina_regular_precios_test = serie_gasolina_regular_precios_test.dropna()
serie_diesel_precios_test = serie_diesel_precios_test.dropna()
serie_gas_lp_precios_test = serie_gas_lp_precios_test.dropna()

# Obtener los datos de validation
serie_gasolina_super_precios_validation = serie_gasolina_super_precios['2023-01-01':'2023-12-31']
serie_gasolina_regular_precios_validation = serie_gasolina_regular_precios['2023-01-01':'2023-12-31']
serie_diesel_precios_validation = serie_diesel_precios['2023-01-01':'2023-12-31']
serie_gas_lp_precios_validation = serie_gas_lp_precios['2023-01-01':'2023-12-31']

# Remplazamos las series por las que tienen los datos de 2024 para abajo
serie_gasolina_super_precios = serie_gasolina_super_precios[:'2022-12-31']
serie_gasolina_regular_precios = serie_gasolina_regular_precios[:'2022-12-31']
serie_diesel_precios = serie_diesel_precios[:'2022-12-31']
serie_gas_lp_precios = serie_gas_lp_precios[:'2022-12-31']

### Se normalizan las series para que las redes neuronales trabajen mejor con ellas. 

In [11]:
# Normalizamos las series
scaler = StandardScaler()
scaler.fit(serie_gasolina_super.values.reshape(-1, 1))
serie_gasolina_super = scaler.transform(serie_gasolina_super.values.reshape(-1, 1))
serie_gasolina_regular = scaler.transform(serie_gasolina_regular.values.reshape(-1, 1))
serie_diesel = scaler.transform(serie_diesel.values.reshape(-1, 1))
serie_gas_lp = scaler.transform(serie_gas_lp.values.reshape(-1, 1))


# Normalizamos las series de importación
scaler_import = StandardScaler()
scaler_import.fit(serie_gasolina_super_import.values.reshape(-1, 1))
serie_gasolina_super_import = scaler_import.transform(serie_gasolina_super_import.values.reshape(-1, 1))
serie_gasolina_regular_import = scaler_import.transform(serie_gasolina_regular_import.values.reshape(-1, 1))
serie_diesel_import = scaler_import.transform(serie_diesel_import.values.reshape(-1, 1))
serie_gas_lp_import = scaler_import.transform(serie_gas_lp_import.values.reshape(-1, 1))

# Normalizamos las series de precios
scaler_precios = StandardScaler()
scaler_precios.fit(serie_gasolina_super_precios.values.reshape(-1, 1))
serie_gasolina_super_precios = scaler_precios.transform(serie_gasolina_super_precios.values.reshape(-1, 1))
serie_gasolina_regular_precios = scaler_precios.transform(serie_gasolina_regular_precios.values.reshape(-1, 1))
serie_diesel_precios = scaler_precios.transform(serie_diesel_precios.values.reshape(-1, 1))

In [12]:
# Normalizamos las series de test
scaler_test = StandardScaler()
scaler_test.fit(serie_gasolina_super_test.values.reshape(-1, 1))
serie_gasolina_super_test = scaler_test.transform(serie_gasolina_super_test.values.reshape(-1, 1))
serie_gasolina_regular_test = scaler_test.transform(serie_gasolina_regular_test.values.reshape(-1, 1))
serie_diesel_test = scaler_test.transform(serie_diesel_test.values.reshape(-1, 1))
serie_gas_lp_test = scaler_test.transform(serie_gas_lp_test.values.reshape(-1, 1))

# Normalizamos las series de importación de test
scaler_import_test = StandardScaler()
scaler_import_test.fit(serie_gasolina_super_import_test.values.reshape(-1, 1))
serie_gasolina_super_import_test = scaler_import_test.transform(serie_gasolina_super_import_test.values.reshape(-1, 1))
serie_gasolina_regular_import_test = scaler_import_test.transform(serie_gasolina_regular_import_test.values.reshape(-1, 1))
serie_diesel_import_test = scaler_import_test.transform(serie_diesel_import_test.values.reshape(-1, 1))
serie_gas_lp_import_test = scaler_import_test.transform(serie_gas_lp_import_test.values.reshape(-1, 1))

# Normalizamos las series de precios de test
scaler_precios_test = StandardScaler()
scaler_precios_test.fit(serie_gasolina_super_precios_test.values.reshape(-1, 1))
serie_gasolina_super_precios_test = scaler_precios_test.transform(serie_gasolina_super_precios_test.values.reshape(-1, 1))
serie_gasolina_regular_precios_test = scaler_precios_test.transform(serie_gasolina_regular_precios_test.values.reshape(-1, 1))
serie_diesel_precios_test = scaler_precios_test.transform(serie_diesel_precios_test.values.reshape(-1, 1))
serie_gas_lp_precios_test = scaler_precios_test.transform(serie_gas_lp_precios_test.values.reshape(-1, 1))


In [13]:
# Normalizamos las series de validation
scaler_validation = StandardScaler()
scaler_validation.fit(serie_gasolina_super_validation.values.reshape(-1, 1))
serie_gasolina_super_validation = scaler_validation.transform(serie_gasolina_super_validation.values.reshape(-1, 1))
serie_gasolina_regular_validation = scaler_validation.transform(serie_gasolina_regular_validation.values.reshape(-1, 1))
serie_diesel_validation = scaler_validation.transform(serie_diesel_validation.values.reshape(-1, 1))
serie_gas_lp_validation = scaler_validation.transform(serie_gas_lp_validation.values.reshape(-1, 1))


# Normalizamos las series de importación de validation
scaler_import_validation = StandardScaler()
scaler_import_validation.fit(serie_gasolina_super_import_validation.values.reshape(-1, 1))
serie_gasolina_super_import_validation = scaler_import_validation.transform(serie_gasolina_super_import_validation.values.reshape(-1, 1))
serie_gasolina_regular_import_validation = scaler_import_validation.transform(serie_gasolina_regular_import_validation.values.reshape(-1, 1))
serie_diesel_import_validation = scaler_import_validation.transform(serie_diesel_import_validation.values.reshape(-1, 1))
serie_gas_lp_import_validation = scaler_import_validation.transform(serie_gas_lp_import_validation.values.reshape(-1, 1))


# Normalizamos las series de precios de validation
scaler_precios_validation = StandardScaler()
scaler_precios_validation.fit(serie_gasolina_super_precios_validation.values.reshape(-1, 1))
serie_gasolina_super_precios_validation = scaler_precios_validation.transform(serie_gasolina_super_precios_validation.values.reshape(-1, 1))
serie_gasolina_regular_precios_validation = scaler_precios_validation.transform(serie_gasolina_regular_precios_validation.values.reshape(-1, 1))
serie_diesel_precios_validation = scaler_precios_validation.transform(serie_diesel_precios_validation.values.reshape(-1, 1))
serie_gas_lp_precios_validation = scaler_precios_validation.transform(serie_gas_lp_precios_validation.values.reshape(-1, 1))


Para que pueda usarse el argoritmo LSTM es necesario transformar la serie en una supervisada, de manera que las observaciones anteriores sean capaces de estimar la siguiente. En este caso vamos a usar el mes anterior para predecir el siguiente por lo que vamos a transformar el arreglo de numpy en una matriz de dos dimensiones en la que la dimensión 1 sea el instante t-1 y la dimensión 2 sea el instante t

In [14]:
from operator import concat


def supervisada(serie,retrasos = 1):
    serie_x = []
    serie_y = []
    for i in range(len(serie)-retrasos):
        valor = serie[i:(i+retrasos),0]
        valor_sig = serie[i+retrasos,0]
        serie_x.append(valor)
        serie_y.append(valor_sig)
    return np.array(serie_x), np.array(serie_y)



In [15]:
# Supervisar series de consumo
# Gasolina Super
x_train_gasolina_super_train, y_train_gasolina_super_train = supervisada(serie_gasolina_super,1)
x_train_gasolina_super_test, y_train_gasolina_super_test = supervisada(serie_gasolina_super_test,1)
x_train_gasolina_super_validation, y_train_gasolina_super_validation = supervisada(serie_gasolina_super_validation,1)

# Gasolina Regular
x_train_gasolina_regular_train, y_train_gasolina_regular_train = supervisada(serie_gasolina_regular,1)
x_train_gasolina_regular_test, y_train_gasolina_regular_test = supervisada(serie_gasolina_regular_test,1)
x_train_gasolina_regular_validation, y_train_gasolina_regular_validation = supervisada(serie_gasolina_regular_validation,1)

# Diesel
x_train_diesel_train, y_train_diesel_train = supervisada(serie_diesel,1)
x_train_diesel_test, y_train_diesel_test = supervisada(serie_diesel_test,1)
x_train_diesel_validation, y_train_diesel_validation = supervisada(serie_diesel_validation,1)

# Gas LP
x_train_gas_lp_train, y_train_gas_lp_train = supervisada(serie_gas_lp,1)
x_train_gas_lp_test, y_train_gas_lp_test = supervisada(serie_gas_lp_test,1)
x_train_gas_lp_validation, y_train_gas_lp_validation = supervisada(serie_gas_lp_validation,1)

### Creamos el modelo.   
#### Tensor Flow
Vamos a utilizar solo una capa LSTM 

In [16]:
lote = 1
unidades =  1
paso = 1
caracteristicas = 1 #es univariada

inputs = keras.layers.Input((paso,caracteristicas))
lstm_out = keras.layers.LSTM(lote)(inputs)
outputs = keras.layers.Dense(1)(lstm_out)



modelo_consumo_gasolina_super = keras.models.Model(inputs=inputs, outputs=outputs)
modelo_consumo_gasolina_regular = keras.models.Model(inputs=inputs, outputs=outputs)
modelo_consumo_diesel = keras.models.Model(inputs=inputs, outputs=outputs)
modelo_consumo_gas_lp = keras.models.Model(inputs=inputs, outputs=outputs)
consumo_models = {
    'gasolina_super': modelo_consumo_gasolina_super,
    'gasolina_regular': modelo_consumo_gasolina_regular,
    'diesel': modelo_consumo_diesel,
    'gas_lp': modelo_consumo_gas_lp
}


# Modelos de importación
modelo_importacion_gasolina_super = keras.models.Model(inputs=inputs, outputs=outputs)
modelo_importacion_gasolina_regular = keras.models.Model(inputs=inputs, outputs=outputs)
modelo_importacion_diesel = keras.models.Model(inputs=inputs, outputs=outputs)
modelo_importacion_gas_lp = keras.models.Model(inputs=inputs, outputs=outputs)
importacion_models = {
    'gasolina_super': modelo_importacion_gasolina_super,
    'gasolina_regular': modelo_importacion_gasolina_regular,
    'diesel': modelo_importacion_diesel,
    'gas_lp': modelo_importacion_gas_lp
}

# Modelos de precios
modelo_precios_gasolina_super = keras.models.Model(inputs=inputs, outputs=outputs)
modelo_precios_gasolina_regular = keras.models.Model(inputs=inputs, outputs=outputs)
modelo_precios_diesel = keras.models.Model(inputs=inputs, outputs=outputs)
modelo_precios_gl = keras.models.Model(inputs=inputs, outputs=outputs)
precios_models = {
    'gasolina_super': modelo_precios_gasolina_super,
    'gasolina_regular': modelo_precios_gasolina_regular,
    'diesel': modelo_precios_diesel,
    'gl': modelo_precios_gl
}


In [17]:
# Compile Models

for key, model in consumo_models.items():
    model.compile(loss='mean_squared_error',optimizer=Adam(learning_rate=0.1), metrics = [RootMeanSquaredError()])

for key, model in importacion_models.items():
    model.compile(loss='mean_squared_error',optimizer=Adam(learning_rate=0.1), metrics = [RootMeanSquaredError()])

for key, model in precios_models.items():
    model.compile(loss='mean_squared_error',optimizer=Adam(learning_rate=0.1), metrics = [RootMeanSquaredError()])



In [18]:
# Train models with 50 epochs
for key, model in consumo_models.items():
    model.fit(x_train_gasolina_super_train.reshape(-1,1,1), y_train_gasolina_super_train, epochs=50, batch_size=1, verbose=0)
    model.save(f'models/consumo_{key}.h5')
    print(f'{key} trained')



gasolina_super trained


gasolina_regular trained


diesel trained


gas_lp trained
